# Analysis Overview

The goal of this notebook will be to provide an overview of the updated analysis of our study results and the choices that went into that anlaysis. A version of this notebook will be included in supplemental materials with directions on how to set up the requisite environment using [Anaconda](https://www.anaconda.com/). A version will also be hosted on [Github](https://github.com/) with an associated [Binder](https://mybinder.org/) envronment for improved acessibility.

*(n.b., this is simply DRAFT v1 -- current accompanying text is placeholder, needs cleaning up / fleshing out)*

---

## Intializiation

The following sections contain handle the requisite setup including the enivironment setup and data import.

In [1]:
# ---------- Imports ----------
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets

import warnings
from copy import deepcopy

In [2]:
# ---------- Setup ----------
# data directory
dataDir = "./data/"

# color maps -- note convert from data specification to labels
cmap_label = ['Traditional', 'Jet', 'Kindlmann', 'Gray']
cmap_str = ['Default', 'Jet', 'Kindlmann', 'Gray']

# additional labels
param_label = ['L*', 'C*', 'h']
indicator_label = ['Cusps (High Curvature)', 'Inflection Points']

### Data Import

#### Color Maps
Here we load the four color maps used in our experiment.

In [3]:
# load all color maps, make accessible by label
cmaps = dict(zip(cmap_label,list(pd.read_csv(dataDir+v.lower()+".csv") for v in cmap_str)))

#### Participant Responses / Records
Here we load the particpant responses from the experiment and specify the outliers, as determined during the quantiative analysis found elsewhere in the supplemental materials.

In [4]:
# import records
records = pd.read_csv(dataDir+"categoryRecords.csv")
records2 = pd.read_csv(dataDir+"boundaryRecords.csv")

# outliers as dictated by our analysis
outliers = ['9229247b-f93a-4bd4-a47a-08ccc1addf2c', 
            '85434358-433b-4dcf-ac84-a87418eed8ef', 
            'c47869d4-cabe-40b9-89f4-74c76451dfe5', 
            '2652ec50-3918-492f-9d65-6210c2ca3031', 
            'cd9cf3f9-29f6-40df-baca-c3e77d71f0be', 
            'cfd3daa0-df19-4714-a47b-bf8d39aa74c5']

Given that our analysis does not appear to show marked differenes between our wording conditions, those records could be analyzed together. For those interested, a merged set of records can be created as follows.

### Convenience Definitions

Below are various function defintions used within the notebook.

In [5]:
# ---------- Convenience Functions ----------
from ConvenienceClasses import SingleLinePlotHandle, Interpolated, ParamControl

# generates figure that plots curves w/ indicators
def plot_curves(cmaps, order=None, indicators=None, legend=False):
    
    # if no labels provided, use keys from required dict
    cmap_labels = list(cmaps.keys())
    if order is not None:
        cmap_labels = [cmap_labels[i] for i in order]
    
    # setup figure
    fig = plt.figure(figsize=(9.5,3.7))
    fig.subplots_adjust(top=0.90, right=0.97, left=0.07, bottom=0.05)

    # setup gridspecs
    gs = gridspec.GridSpec(5, len(cmaps.keys()), height_ratios=[2,4,4,4,1])
    
    # convenience switches
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    colors_switch = {'L*':['#0E6089','#5E50A3'], 'C*':['#0E6089', '#DB7723'], 'h':['#992B65', '#1C9B37']}

    # for each color map
    for i in range(len(cmaps.keys())):
        
        # grab colormap
        colormap = cmaps[cmap_labels[i]]
        
        # convenience switch
        colormap_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}

        # create image
        rgb = colormap[['r','g','b']].values
        rgb.shape = (1,rgb.shape[0],rgb.shape[1])
        rgb_image = np.repeat(rgb, 50, axis=0)
        
        # plot image
        ax = plt.subplot(gs[0,i])
        ax.imshow(rgb_image, interpolation='nearest', aspect='auto', extent=[0.0,1.0,0.0,1.0])
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(True)
        ax.tick_params(labelbottom=False)  
        ax.set_title(cmap_labels[i])

        # store axes
        ax_handles = []

        # setup plots
        #   Luminance
        ax = plt.subplot(gs[1,i])
        ax.plot(colormap.x, colormap_switch['L*'], color='#333333')
        ax.set_xlim([0,1])
        ax.set_ylim(ylims_switch['L*'])
        ax.tick_params(labelbottom=False)  
        ax_handles.append(ax)

        #   Chroma    
        ax = plt.subplot(gs[2,i])
        ax.plot(colormap.x, colormap_switch['C*'], color='#333333')
        ax.set_xlim([0,1])
        ax.set_ylim(ylims_switch['C*'])
        ax.tick_params(labelbottom=False)
        ax_handles.append(ax)

        #   Hue    
        ax = plt.subplot(gs[3,i])
        ax.plot(colormap.x, colormap_switch['h'], color='#333333')
        ax.set_xlim([0,1])
        ax.set_ylim(ylims_switch['h'])
        ax.set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        ax.set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
        ax_handles.append(ax)
        
        # if indicators available
        if indicators is not None:
            # grab color map label
            c = cmap_labels[i]
            # for parameter axes
            for j in range(len(ax_handles)):
                p = param_label[j]
                ax = ax_handles[j]
                cc = colors_switch[p]
                
                # draw indicators
                for xc in indicators[c][p]['Cusps (High Curvature)']:
                    ax.axvline(x=xc, color=cc[0], linestyle='--', alpha=0.55)
                
                for xc in indicators[c][p]['Inflection Points']:
                    ax.axvline(x=xc, color=cc[1], linestyle='--', alpha=0.55)
                    
        # add labels to right hand side of plots
        if (i==0):
            ax_handles[0].set_ylabel(r'L$^{*}$', rotation='horizontal', fontsize=14, va='center', ha='right', labelpad=3)
            ax_handles[1].set_ylabel(r'C$^{*}$', rotation='horizontal', fontsize=14, va='center', ha='right', labelpad=3)
            ax_handles[2].set_ylabel('h',  rotation='horizontal', fontsize=14, va='center', ha='right', labelpad=13)
    
    return fig

#  indicator derivation plots
def plotParam(figure, colormap, param='L*', xs=np.linspace(0,1,10000)):
    # setup gridspecs
    gs = gridspec.GridSpec(4, 1, height_ratios=[1.5,4,4,4])
    
    # switch dicts
    colormap_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    
    # generate spline
    current_param = Interpolated(colormap.x, colormap_switch[param], 0.5)
    
    # create image
    rgb = colormap[['r','g','b']].values
    rgb.shape = (1,rgb.shape[0],rgb.shape[1])
    rgb_image = np.repeat(rgb, 50, axis=0)
    
    axes = []
    # plot image
    ax = plt.subplot(gs[0,0])
    im_obj = ax.imshow(rgb_image, interpolation='nearest', aspect='auto')
    ax.get_yaxis().set_visible(False)
    ax.tick_params(labelbottom=False)
    axes.append(ax)
    
    # plot spline
    ax = plt.subplot(gs[1,0])
    l, = ax.plot(xs, current_param.spline(xs), color='C0')
    ax.set_xlim([0,1])
    ax.tick_params(labelbottom=False)
    ax.set_ylim(ylims_switch[param])
    if (param is 'h'):
        ax.set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        ax.set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    # plot |grad|
    ax = plt.subplot(gs[2,0])
    l, = ax.plot(xs, np.abs(current_param.grad(xs)), color='C0')
    ax.set_xlim([0,1])
    ax.tick_params(labelbottom=False)
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    # plot |curv|
    ax = plt.subplot(gs[3,0])
    l, = ax.plot(xs, np.abs(current_param.curv(xs)), color='C0')
    ax.set_xlim([0,1])
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    return (axes, current_param, im_obj)

# Sigma Sensitivity Analysis
def sigma_senstitivity(colormap, param_str, indicator_str, sigma_values=[], thresholds=[], tol=0.005):
        
    colormap_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    current_param = Interpolated(colormap.x, colormap_switch[param_str], 0)

    indicator_locations = []
    for i in range(len(sigma_values)):
        current_param.sigma = sigma_values[i]
        if indicator_str == indicator_label[0]:
            _list = current_param.getHighCurv(thresholds[i])
        elif indicator_str == indicator_label[1]:
            _list = current_param.getInflectionPoints(thresholds[i])
        else:
            _list = []
        indicator_locations.append(_list)

    indicator_sets = []
    iterators = np.array([iter(l) for l in indicator_locations])
    current = np.array([next(i, np.NaN) for i in iterators])

    nan_mask = np.isnan(current)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore") #ignore NaN runtime warning -- we're relying on propogating NaNs for stop condition
        while not all(nan_mask):
            mask = (current - min(current)) < tol    
            push = current.copy()
            push[~nan_mask & ~mask] = np.NaN
            indicator_sets.append(push)
            current[~nan_mask & mask] = np.array([next(i, np.NaN) for i in iterators[~nan_mask & mask]])
            nan_mask = np.isnan(current)

    df = pd.DataFrame(np.transpose(np.array(indicator_sets)))
    df.insert(0, 'sigma', sigma_values)
    df.insert(1, 'threshold', thresholds)
    
    return (df)


In [6]:
# Define Indicators

# the original indicators
indicators_full = {
    'Traditional': {
        'L*': {
            'Cusps (High Curvature)': [0.25098,  0.74902],
            'Inflection Points': [0.214502]
        },
        'C*': {
            'Cusps (High Curvature)': [0.25098,  0.74902],
            'Inflection Points': [0.096852,  0.244349,  0.256319,  0.355024,  0.638564,  0.759411,  0.913586]
        },
        'h': {
            'Cusps (High Curvature)': [],
            'Inflection Points': [0.205089,  0.832291]
        }
    },
    'Jet': {
        'L*': {
            'Cusps (High Curvature)': [0.109804,  0.376471,  0.639216,  0.890196],
            'Inflection Points': [0.012758,  0.326049,  0.348109,  0.361094,  0.672307,  0.902486,  0.919723,  0.987242]
        },
        'C*': {
            'Cusps (High Curvature)': [0.109804,  0.376471,  0.639216,  0.890196],
            'Inflection Points': [0.007324,  0.01144,  0.012927,  0.099824,  0.102156,  0.221824,  0.369223,  0.383768,  0.419042,  0.646978,  0.647978,  0.651844,  0.816664,  0.992781]
        },
        'h': {
            'Cusps (High Curvature)': [0.341176,  0.376471,  0.639216,  0.658824],
            'Inflection Points': [0.347344,  0.652104,  0.728837]
        }
    },
    'Kindlmann': {
        'L*': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        },
        'C*': {
            'Cusps (High Curvature)': [0.098039,  0.2,  0.298039 , 0.596078,  0.698039,  0.8,  0.811765],
            'Inflection Points': [0.11162,  0.146164,  0.309554,  0.474843,  0.80841]
        },
        'h': {
            'Cusps (High Curvature)': [0.007843,  0.094118,  0.113725,  0.2,  0.301961,  0.32549,  0.701961,  0.878431,  0.901961,  0.988235],
            'Inflection Points': [0.107047,  0.210065,  0.356393,  0.720725,  0.885858,  0.990418]
        }
    },
    'Gray': {
        'L*': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        },
        'C*': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        },
        'h': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        }
    }
}

# reduced, removes numeric artifacts
indicators = deepcopy(indicators_full)
indicators['Traditional']['C*']['Inflection Points'] = [0.096852,  0.355024,  0.638564,  0.913586]
indicators['Jet']['L*']['Inflection Points'] = [0.326049,  0.348109,  0.672307,  0.919723]
indicators['Jet']['C*']['Inflection Points'] = [0.221824,  0.419042,  0.816664]
indicators['Jet']['h']['Inflection Points'] = [0.728837]
indicators['Kindlmann']['C*']['Cusps (High Curvature)'] = [0.098039,  0.2,  0.298039 , 0.596078,  0.698039,  0.8]
indicators['Kindlmann']['C*']['Inflection Points'] = [0.146164,  0.474843]
indicators['Kindlmann']['h']['Inflection Points'] = [0.356393]

#### Notebook Formatting

In [7]:
%%html
<style> table{float:left} </style>

---

## Indicator Defintions 

We'll start with an overview of our finalized indicator set. 

The high level overview description is basically that based on our H2 hypotheses (todo: summarize), we are interested in cusps and inflection points in the CIELCH luminance, chroma, and hue profiles.

The resulting indicator sets used in our analysis are shown below and include:

- **L*** / **C***  Cusps (*via high curvature*)
- **h**  Cusps (*via high curvature*)
- **L*** Inflection Points
- **C*** Inflection Points
- **h**  Inflection Points

In [8]:
# plot curves
fig_curves = plot_curves(cmaps, order=[0,2,3,1], indicators=indicators)
fig_curves.show()
#fig_curves.savefig(outfile, dpi=150)

FigureCanvasNbAgg()

## Derivation

For consistency / in line with how color maps are often applied in practice, each of the color maps we tested were created as 256 value color lookup tables. For each of color value in these tables we take the corresponding CIELCH luminance, chroma, and hue value. We use these values to create cubic-spline interpolating polynomials that approximate the underlying CIELCH profiles. 

To find cusps, we look for locations of *high curvature* in the interpolating polynomials. We use the roots of the third derivative of these interpolating polynomials to locate the local maxima / minima in the second derivative, which represents curvature. Taking the absolute value of second derivive provides curvature magnitude, which we can theshold to generate a set of local maxima with arbitrarily high curvature magnitude.

The inflection points are derived similarly, only starting with the roots of the second derivative (i.e., the locations of zero curvature) and thresholding based on gradient magnitude.

Becuase numerical approximations are known to be sensitive to noise, we also include a smoothing parameter $\sigma$, which applies Gaussian smoothing to the luminance, chroma, and hue values from the color scale before creating the cubic-spline interpolating polynomials.

The specific smoothing and thesholding parameters we used can be all be found in the table below.

| Color Map   | Indicator              | Parameter | $\sigma$ | $\tau$ |
|------------:|:-----------------------|:---------|:--------|:------|
| Traditional |High Curvature         | L*        | 0       | 90000 |
| Traditional |High Curvature         | C*        | 0       | 37000 |
| Traditional |Inflection Points      | L*        | 0.6     | 295   |
| Traditional |Inflection Points      | C*        | 1.0     | 135   |
| Traditional |Inflection Points      | h         | 0       | 3     |
| Jet         |High Curvature         | L*        |       0 |  60000|
| Jet         |High Curvature         | C*        |       0 |  80000|
| Jet         |High Curvature         | h         |       0 |    990|
| Jet         |Inflection Points      | L*        |     1.2 |    158|
| Jet         |Inflection Points      | C*        |     0.8 |    100|
| Jet         |Inflection Points      | h         |     1.0 |      6|
| Kindlmann   |High Curvature         | C*        |     1.3 |  18000|
| Kindlmann   |High Curvature         | h         |     1.7 |    270|
| Kindlmann   |Inflection Points      | C*        |     1.5 |    370|
| Kindlmann   |Inflection Points      | h         |     2.5 |    3.4|

We've also included an interactive tool that illustrates the derivation process for each indicator set.

In [9]:
# ------------ INDICATOR DERIVATION TOOL ------------
from matplotlib.ticker import AutoLocator, ScalarFormatter

plt.ioff() # turns interactive mode off
plt.clf()  # clears current figure

# ------------ SMOOTHING / THRESHOLDING PARAMETERS ------------

# dictionary for switching settings
# returns (sigma, gradient_threshold, curvature_threshold)
indicator_settings = {
    'Traditional': {
        'L*': {
            'Cusps (High Curvature)': (0, None, 90000),
            'Inflection Points': (0.6, 295, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, 37000),
            'Inflection Points': (1.0, 135, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, 3, None)
        }
    },
    'Jet': {
        'L*': {
            'Cusps (High Curvature)': (0, None, 60000),
            'Inflection Points': (1.2, 158, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, 80000),
            'Inflection Points': (0.8, 100, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, 990),
            'Inflection Points': (1.0, 6, None)
        }
    },
    'Kindlmann': {
        'L*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'C*': {
            'Cusps (High Curvature)': (1.3, None, 18000),
            'Inflection Points': (1.5, 370, None)
        },
        'h': {
            'Cusps (High Curvature)': (1.7, None, 270),
            'Inflection Points': (2.5, 3.4, None)
        }
    },
    'Gray': {
        'L*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        }
    }
}

# ------------ INITIALIZE CONTROLS, PT. 1 ------------

# initialize controls
cmap_select = widgets.Select(
                                options=cmap_label,
                                value='Traditional',
                                rows=4,
                                description='Color Map:',
                                disabled=False
                             )

param_select = widgets.Select(
                                        options=param_label,
                                        value='L*',
                                        rows=3,
                                        description='Channel:',
                                        disabled=False
                             )

indicator_select = widgets.Select(
                                        options=indicator_label,
                                        value='Cusps (High Curvature)',
                                        rows=2,
                                        description='Indicators:',
                                        disabled=False
                             )

reset_button = widgets.Button(description='reset')

# ------------ SETUP FIGURE ------------

# setup figure
fig_edgePlot = plt.figure(figsize=(3.5,5))# 11,4
fig_edgePlot.subplots_adjust(top=0.95, right=0.95, left=0.17, bottom=0.05)

xs = np.linspace(0,1,10000) # values for plotting functions

# plot
axes, current_param, im_obj = plotParam(fig_edgePlot, cmaps[cmap_select.value], param_select.value, xs=xs)

# fig_edgePlot.show() will display a static plot, for interactive see below

# ------------ INITIALIZE CONTROLS, PT. 2 ------------

current_control = ParamControl(current_param, axes[1], axes[2], axes[3])

# ------------ UPDATE SCHEMES ------------
    
def update(figure, xs=[], control=None, grad=False, curv=False): 
        
    if control is not None: 
        if   grad: control.update_grad()
        elif curv: control.update_curv()
        else:      control.update_full(xs)
        
        figure.canvas.draw_idle()
        figure.canvas.flush_events()
        
def override(figure, axes, colormap, param_str, controls, im_obj=None):
    
    # override data
    param_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    current_param.override(colormap.x, param_switch[param_str], current_param.sigma)
    
    # override ylims and ticks
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    axes[1].set_ylim(ylims_switch[param_str])
    if (param_str == 'h'):
        axes[1].set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        axes[1].set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
    else:
        axes[1].yaxis.set_major_locator(AutoLocator())
        axes[1].yaxis.set_major_formatter(ScalarFormatter())
        
    # override indicator colors
    colors_switch = {'L*':['#0E6089','#5E50A3'], 'C*':['#0E6089', '#DB7723'], 'h':['#992B65', '#1C9B37']}
    cc = colors_switch[param_str]
    controls.setColors(cc[0], cc[1])
    
    # re-draw
    update(figure, xs=xs, control=controls)

def update_cmap(change, im_obj):
    
    # create new image
    colormap = cmaps[cmap_select.value]
    
    # create new image
    rgb = colormap[['r','g','b']].values
    rgb.shape = (1,rgb.shape[0],rgb.shape[1])
    rgb_image = np.repeat(rgb, 50, axis=0)
    im_obj.set_data(rgb_image)
    
    # override plots
    override(fig_edgePlot, axes, colormap, param_select.value, current_control)
    
    #reset indicators
    update_indicators(current_control)  # indicator_select.value = None

def update_param(change):
    
    # override plots
    override(fig_edgePlot, axes, cmaps[cmap_select.value], param_select.value, current_control)
    
    # reset indicators
    update_indicators(current_control)  # indicator_select.value = None

def update_indicators(controls):
    
    s, g_thresh, c_thresh = indicator_settings[cmap_select.value][param_select.value][indicator_select.value]
    
    controls.sigma_control.value = s
    controls.gradient_threshold.value = g_thresh if g_thresh is not None else controls.gradient_threshold.max
    controls.curvature_threshold.value = c_thresh if c_thresh is not None else controls.curvature_threshold.max

# ------------ LINK CONTROLS ------------

current_control.sigma_control.observe(       lambda change : update(fig_edgePlot, control=current_control, xs=xs),     names='value' )
current_control.curvature_threshold.observe( lambda change : update(fig_edgePlot, control=current_control, curv=True), names='value' )
current_control.gradient_threshold.observe(  lambda change : update(fig_edgePlot, control=current_control, grad=True), names='value' )

cmap_select.observe( lambda change : update_cmap(change, im_obj), names='value')
param_select.observe(update_param, names='value')
indicator_select.observe(lambda change : update_indicators(current_control), names='value')

reset_button.on_click(lambda b : update_indicators(current_control))

# ------------ DISPLAY ------------

# update initial controls state
update_indicators(current_control)

# display
widgets.HBox([widgets.VBox([cmap_select,param_select,indicator_select,current_control.widget,reset_button]), fig_edgePlot.canvas])

### Sensitivity Analysis

I'll worry about expanding the writeup on this later, but this section will delve into:

1. Why we chose the smoothing and thesholding paramters that we did
2. Why we manually removed a subset of indicators

Looking at the sensitivity analysis we could get away with picking a single smoothing value for both the Traditional and Jet color maps. With the Kindlmann color map, I'd be a little more concerned due to the sheer amount of smoothing required to isolate the inflectction points in hue... As it is, though I think keeping the varying parmeters is fine as long as we're providing the justification like below.

- *(For readability we may want to reduce this to a few illustative examples and include a more extensive set of comparisons in a seperate notebook)*
- *(Also may want to plot some of these comparisons...)*

#### Traditional Rainbow

The high curvature indicators tend to be fairly invariant to a small amount of gaussian smoothing

In [10]:
sigma_senstitivity(cmaps['Traditional'], 'L*', 'Cusps (High Curvature)', sigma_values=[0, 0.6], thresholds=[60000, 60000])

,sigma,threshold,0,1
0,0.0,60000,0.25098,0.74902
1,0.6,60000,0.25098,0.74902


We can also see that the locations of high curvature in luminance and chroma are the same

In [11]:
sigma_senstitivity(cmaps['Traditional'], 'C*', 'Cusps (High Curvature)', sigma_values=[0, 0.8], thresholds=[30000,15000])

,sigma,threshold,0,1
0,0.0,30000,0.25098,0.74902
1,0.8,15000,0.25098,0.74902


The inflection point derivations on the other hand are not as inavraint to the gaussian smoothing...

In [12]:
sigma_senstitivity(cmaps['Traditional'], 'L*', 'Inflection Points', sigma_values=[0, 0.6, 0.8], thresholds=[295, 295, 295])

,sigma,threshold,0,1,2,3
0,0.0,295,0.214402,0.233036,0.235872,0.243943
1,0.6,295,0.214502,NaN,NaN,NaN
2,0.8,295,0.214556,NaN,NaN,NaN


That being said, it's clear some these indicators have significantly more varaince than others (e.g. 3, 4, and 7 below). As these indicators all approximate the loactions of high curvature, however, they wind being removed from our analysis.

Because we use interpolating polynomials to approximate the undelrying L*, C*, and h profiles, the cusps in those profile are essentially super high spatial frequency changes in concavity. This necessitates there being zero crossings in the gradient nearby, which would not ne there if we had modeled the profiles as a peicewise polynomial, for example. This makes them artifacts of the numerical processes we chose, not quantities of interest.

In [13]:
sigma_senstitivity(cmaps['Traditional'], 'C*', 'Inflection Points', sigma_values=[0, 0.8], thresholds=[135, 135])

,sigma,threshold,0,1,2,3,4,5,6,7,8
0,0.0,135,0.002037,0.008302,0.096767,0.247227,0.253740,0.355011,0.638593,0.75501,0.913557
1,0.8,135,NaN,NaN,0.096822,0.244724,0.255113,0.355019,0.638575,0.75749,0.913576


#### Jet

Again we see invariance in the high curvature indicators...

In [14]:
sigma_senstitivity(cmaps['Jet'], 'C*', 'Cusps (High Curvature)', sigma_values=[0, 0.6, 0.8, 1.0, 1.2], thresholds=[80000,50000,27000,27000,24000])

,sigma,threshold,0,1,2,3
0,0.0,80000,0.109804,0.376471,0.639216,0.890196
1,0.6,50000,0.109804,0.376471,0.639216,0.890196
2,0.8,27000,0.109804,0.376471,0.639216,0.890196
3,1.0,27000,0.109804,0.376471,0.639216,0.890196
4,1.2,24000,0.109804,0.376471,0.639216,0.890196


In [15]:
sigma_senstitivity(cmaps['Jet'], 'C*', 'Inflection Points', sigma_values=[0.65,0.8,1.0, 1.1], thresholds=[100,100,100,100])

,sigma,threshold,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.65,100,0.007029,0.011348,0.013345,0.100383,0.102187,0.221806,0.369576,0.382988,0.419001,0.646387,0.648714,0.652351,0.816658,0.90163,0.902898,0.993096
1,0.80,100,0.007324,0.011440,0.012927,0.099824,0.102156,0.221824,0.369223,0.383768,0.419042,0.646978,0.647978,0.651844,0.816664,NaN,NaN,0.992781
2,1.00,100,0.010918,NaN,NaN,NaN,NaN,0.221855,0.368720,0.385938,0.419110,0.650843,NaN,NaN,0.816674,NaN,NaN,0.992168
3,1.10,100,0.011598,NaN,NaN,NaN,NaN,0.221873,0.368304,0.387276,0.419149,0.650844,NaN,NaN,0.816680,NaN,NaN,0.989277


With the potential L* Inflection Points (see 5/6, 10, 14 below) the varaiation due to smoothing is more pronounced

In [16]:
sigma_senstitivity(cmaps['Jet'], 'L*', 'Inflection Points', sigma_values=[0.8, 1.1], thresholds=[158,158], tol=0.004)

,sigma,threshold,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.8,158,0.007324,0.011440,0.012927,0.099826,0.102156,0.329803,0.346421,0.365157,0.648553,0.652901,0.669380,0.897623,0.900496,0.904003,0.916876,0.987073,0.98856,0.992676
1,1.1,158,NaN,0.011598,NaN,NaN,NaN,0.326636,0.347749,0.361603,NaN,NaN,0.670879,NaN,0.902313,NaN,0.918847,0.988402,NaN,NaN


In [17]:
sigma_senstitivity(cmaps['Jet'], 'L*', 'Cusps (High Curvature)', sigma_values=[0,1.1], thresholds=[60000,14000])

,sigma,threshold,0,1,2,3
0,0.0,60000,0.109804,0.376471,0.639216,0.890196
1,1.1,14000,0.109804,0.376471,0.639216,0.890196


#### Kindlmann

In [18]:
sigma_senstitivity(cmaps['Kindlmann'], 'h', 'Cusps (High Curvature)', sigma_values=[1.7,2.5], thresholds=[270,180])

,sigma,threshold,0,1,2,3,4,5,6,7,8,9,10,11
0,1.7,270,0.007843,0.094118,0.113725,0.200000,0.301961,0.32549,NaN,0.701961,0.878431,0.901961,NaN,0.988235
1,2.5,180,NaN,0.094118,0.117647,0.196078,0.301961,NaN,0.431373,0.701961,0.874510,0.901961,0.980392,NaN


(2) is the only non-confounded inflection point / the only one included in our analysis

In [19]:
sigma_senstitivity(cmaps['Kindlmann'], 'h', 'Inflection Points', sigma_values=[2.5], thresholds=[3.4])

,sigma,threshold,0,1,2,3,4,5
0,2.5,3.4,0.107047,0.210065,0.356393,0.720725,0.885858,0.990418


### Removing Artifacts

The indicator derivation procedures outlines above only get us so far. Based on our choices of smoothing and threshlding parameters we are left with indicator sets that look like:

In [20]:
plot_curves(cmaps, order=[0,2,3,1], indicators=indicators_full).show()

FigureCanvasNbAgg()

As mentioned in the sensitivity analysis a number of these indicators can be traced to numeric artifacts. When these are removed we are left with the following, which is what we use in our anlysis.

*(needs significant expansion -- aside from spurious kindlamnn indicator, the discussion is laregly tied to the sensitivity anlaysis discussion, which is also incomplete)*

In [21]:
plot_curves(cmaps, order=[0,2,3,1], indicators=indicators).show()

FigureCanvasNbAgg()

---

That's the end of the planned document. Below are details regarding remaining todo items / edits.  

### ToDo List

- Flesh out placeholder descriptions into proper story 